# 画像で機械学習するための準備（処理時間1分以内）
* Amazon SageMaker はあまり関係ありません


* このノートブックで行うこと
    1. yan lecun 様の[サイト](http://yann.lecun.com/exdb/mnist/)より mnist のデータセットをノートブックインスタンスにダウンロード
    2. バイナリデータから画像データを１枚ずつ png ファイルに出力
    3. バイナリデータからラベルデータを npy ファイルに出力
    4. zip ファイルに固めて S3 にアップロード    
         zip ファイルに固める理由はバラバラの画像ファイルだと転送速度が遅くなるため、固めて転送して演算するコンピューティングリソースで解凍する

![](media/0_data_preparation.png)

In [ ]:
import sagemaker
print(f'Current sagemaker Version ={sagemaker.__version__}')

上記セルを実行して、SageMaker Python SDK Version が 1.xx.x の場合、以下のセルのコメントアウトを解除してから実行してください。実行が完了したら、上にあるメニューから [Kernel] -> [Restart] を選択してカーネルを再起動してください。

再起動が完了したら、このノートブックの一番上のセルから再度実行してください。その場合、以下のセルを実行する必要はありません。

In [ ]:
# pip install -U --quiet "sagemaker==2.20.0"

## 各種設定

In [ ]:
name = '<type your name>'

In [ ]:
import urllib.request, gzip, numpy as np, sagemaker, datetime, yaml, os, shutil
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm

url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}
# 様々な識別子を一意にするためにタイムスタンプを利用する
timestamp = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d%H%M%S')
print(f'timestamp: {timestamp}')

## データをノートブックインスタンスにダウンロード

In [ ]:
%%time

dataset_dir = './mnist/'    #画像ファイルとラベルを保存するディレクトリ

os.makedirs(dataset_dir, exist_ok=True)

for v in key_file.values():
    file_path = dataset_dir + '/' + v
    # データのダウンロード
    urllib.request.urlretrieve(url_base + v, file_path)

In [ ]:
!ls  {dataset_dir}

## mnistのバイナリデータを numpy 配列にする
ついでにラベルデータを事前に one-hot encoding しておく

In [ ]:
%%time

file_path = dataset_dir + key_file['train_img']
with gzip.open(file_path, 'rb') as f:
    train_x = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28,28)
file_path = dataset_dir + key_file['train_label']
with gzip.open(file_path, 'rb') as f:
    train_y = np.frombuffer(f.read(), np.uint8, offset=8)
train_y = np.identity(10)[train_y]


file_path = dataset_dir + key_file['test_img']
with gzip.open(file_path, 'rb') as f:
    test_x = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28,28)
file_path = dataset_dir + key_file['test_label']
with gzip.open(file_path, 'rb') as f:
    test_y = np.frombuffer(f.read(), np.uint8, offset=8)
test_y = np.identity(10)[test_y]

## numpy array を 1 枚ずつ png ファイルに出力する
* [前処理](1_preprocess_kick.ipynb)で hist 平坦化したのち、再度 npy ファイルに変換するため

In [ ]:
%%time

# ローカルへ保存
base_dir = './dataset/'
train_x_dir = base_dir + 'train_x/'
test_x_dir = base_dir + 'test_x/'

os.makedirs(train_x_dir, exist_ok=True)
os.makedirs(test_x_dir, exist_ok=True)

for i in tqdm(range(train_x.shape[0])):
    Image.fromarray(train_x[i,:,:]).save(train_x_dir + str(i).zfill(5) + ".png")

for i in tqdm(range(test_x.shape[0])):
    Image.fromarray(test_x[i,:,:]).save(test_x_dir + str(i).zfill(5) + ".png")

np.save(base_dir + 'train_y.npy',train_y)
np.save(base_dir + 'test_y.npy',test_y)

In [ ]:
!ls -l {base_dir}

In [ ]:
!ls -l {base_dir}train_x | tail -n5

In [ ]:
!ls -l {base_dir}test_x | tail -n5

## zip ファイルに固める
* 前処理をする際に zip ファイルに固めたほうが S3 から前処理を行うコンテナへの転送効率が高い（細切れのファイルだと転送に時間がかかる）
* 解凍処理も前処理に含める

In [ ]:
zip_file = shutil.make_archive('./dataset', 'zip', root_dir='./dataset/')
print(zip_file)

## S3 にアップロードする際、一意の URI を指定したいため、設定ファイルの名前を取得

## SageMaker SDK を利用して 作成した dataset.zip を S3 にアップロード
* 通常 S3 にアップロードをする際は [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#client) や [aws cli](https://aws.amazon.com/jp/cli/) を利用する
* SageMaker SDK はデータサイエンティストのための SDK で boto3 や aws cli などの学習コストを増やさないよう, SageMaker で完結させられる = S3 にアップロードできる機能も持っている
* upload_data メソッドを利用することで SageMaker のデフォルトバケット(sagemaker-{region}-{account} に 1 行のコードでアップロードできる

In [ ]:
# S3 に dataset.zip をアップロード
prefix = f'sagemaker-handson-{name}/dataset-{timestamp}'
zip_dataset_s3_uri = sagemaker.session.Session().upload_data(path=zip_file, key_prefix=prefix)

In [ ]:
# アップロード先などを次のノートブックに引き継ぐため yaml に出力
with open("./setting.yaml", mode='w') as f:
    f.write('name: ' + name +'\n')
    f.write('zip_dataset_s3_uri: ' + zip_dataset_s3_uri + '\n')
    f.write('timestamp: ' + timestamp + '\n')

with open('./setting.yaml', 'r') as yml:
    config = yaml.load(yml)
print(config)